In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import typing
import glob
import time
import datetime
import numpy as np
%config Completer.use_jedi = False

import tensorflow as tf
import tensorflow.keras as keras

# Unknownerror, cudnn 어쩌고저쩌고 에러
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [2]:
print(tf.__version__)

2.5.0


In [3]:
import sys
import random
import soundfile as sf
sys.getsizeof

<function sys.getsizeof>

In [4]:
import multiprocessing

In [5]:
import import_ipynb
from CV_Tasnet_CustomTraining import *

importing Jupyter notebook from CV_Tasnet_CustomTraining.ipynb


In [6]:
import import_ipynb
from CV_Tasnet_Model import *

importing Jupyter notebook from CV_Tasnet_Model.ipynb
tf.__version__: 2.5.0
Sample encoder outputs shape: (8, 1999, 512)
Before blocks shape: (8, 1999, 128)
Total block numbers: 24
Number 0 block done	Number 1 block done	Number 2 block done	Number 3 block done	Number 4 block done	Number 5 block done	Number 6 block done	Number 7 block done	Number 8 block done	Number 9 block done	Number 10 block done	Number 11 block done	Number 12 block done	Number 13 block done	Number 14 block done	Number 15 block done	Number 16 block done	Number 17 block done	Number 18 block done	Number 19 block done	Number 20 block done	Number 21 block done	Number 22 block done	Number 23 block done	
After blocks outputs shape: (8, 1999, 128)
(8, 1999, 1024)
masks.shape: (8, 2, 1999, 512)
Separation_outputs.shape: (8, 2, 1999, 512)
Final outputs shape: (8, 16000, 2)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Sh

conv_block_17 (ConvBlock)       ((8, 1999, 128), (8, 201216      conv_block_16[0][1]              
__________________________________________________________________________________________________
tf.__operators__.add_17 (TFOpLa (8, 1999, 128)       0           tf.__operators__.add_16[0][0]    
                                                                 conv_block_17[0][0]              
__________________________________________________________________________________________________
conv_block_18 (ConvBlock)       ((8, 1999, 128), (8, 201216      conv_block_17[0][1]              
__________________________________________________________________________________________________
tf.__operators__.add_18 (TFOpLa (8, 1999, 128)       0           tf.__operators__.add_17[0][0]    
                                                                 conv_block_18[0][0]              
__________________________________________________________________________________________________
conv_block

In [29]:
data_base_dir = "C:/Users/Phil/anaconda3/envs/LibriMix/storage_dir/Libri2Mix/wav8k/min"

train_input_path = 'train-360/mix_both'
train_target_paths = ['train-360/s1', 'train-360/s2']

val_input_path = 'dev/mix_both'
val_target_paths = ['dev/s1', 'dev/s2']

test_input_path = 'test/mix_both'
test_target_paths = ['test/s1', 'test/s2']

ext = '/*.wav'

In [30]:
# Store a path
train_input_dir = os.path.join(data_base_dir, train_input_path)
train_target_dirs = [os.path.join(data_base_dir, train_path) 
                     for train_path in train_target_paths]

val_input_dir = os.path.join(data_base_dir, val_input_path)
val_target_dirs =[os.path.join(data_base_dir, val_path) 
                  for val_path in val_target_paths]

test_input_dir = os.path.join(data_base_dir, test_input_path)
test_target_dirs = [os.path.join(data_base_dir, test_path)
                   for test_path in test_target_paths]

In [31]:
# Find every files in a selected path
train_input_files = glob.glob(train_input_dir + ext)
train_target_files = [glob.glob(train_path + ext) for train_path in train_target_dirs]

val_input_files = glob.glob(val_input_dir + ext)
val_target_files = [glob.glob(val_path + ext) for val_path in val_target_dirs]

test_input_files = glob.glob(test_input_dir + ext)
test_target_files = [glob.glob(test_path + ext) for test_path in test_target_dirs]

In [32]:
print('Number of training files:', len(train_input_files))
for i in [len(j) for j in train_target_files]:
    assert len(train_input_files) == i

print('Number of validation files:', len(val_input_files))
for i in [len(j) for j in val_target_files]:
    assert len(val_input_files) == i

print('Number of test files:', len(test_input_files))
for i in [len(j) for j in test_target_files]:
    assert len(test_input_files) == i

Number of training files: 50800
Number of validation files: 3000
Number of test files: 3000


In [33]:
import multi_sf_blocks

In [34]:
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [35]:
model_base_dir = "C:/Users/Phil/anaconda3/envs/CTnet"
checkpoint_path = "checkpoints/cp-{epoch:03d}.ckpt"
saving_path = os.path.join(model_base_dir, checkpoint_path)
saving_path

'C:/Users/Phil/anaconda3/envs/CTnet\\checkpoints/cp-{epoch:03d}.ckpt'

In [36]:
model_callback = keras.callbacks.ModelCheckpoint(filepath=saving_path,
                                                 verbose=1,
                                                 save_weights_only=True,
                                                 save_freq='epoch')
callbacks = keras.callbacks.CallbackList([model_callback], model=cv_tasnet)

In [37]:
log_path = os.path.join(model_base_dir, "logs")
summary_writer = tf.summary.create_file_writer(log_path)
log_path

'C:/Users/Phil/anaconda3/envs/CTnet\\logs'

In [42]:
## tf.function 쓰면 안 됨
def train_step(input_batch, target1_batch, target2_batch):
    with tf.GradientTape() as tape:
        predicted_batch = cv_tasnet(input_batch, training=True)
        loss = calculate_sisnr(predicted_batch, tf.cast(tf.concat([target1_batch, target2_batch], axis=-1), dtype=tf.float32),
                              BATCH_SIZE=BATCH_SIZE, num_spks=num_spks)
    trainable_vars = cv_tasnet.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    cv_tasnet.optimizer.apply_gradients(zip(gradients, trainable_vars))
    
    return {'train_sisnr': loss}

In [43]:
def test_step(input_batch, target1_batch, target2_batch):
    predicted_batch = cv_tasnet(input_batch, training=False)
    loss = calculate_sisnr(predicted_batch, tf.cast(tf.concat([target1_batch, target2_batch], axis=-1), dtype=tf.float32),
                          BATCH_SIZE=BATCH_SIZE, num_spks=num_spks)
    return {'val_sisnr': loss}

In [49]:
EPOCHS = 3
per_step = 1500 # 램과 상관이 있음
N_steps = len(train_input_files) // per_step
val_N_steps = len(val_input_files) // per_step

if __name__ == '__main__':
    train_metrics = []
    val_metrics = []
    for epoch in range(EPOCHS):
        print('################### Start of Epoch Num {}, out of {} Epochs ###################'.format(epoch+1, 
                                                                                               EPOCHS))
        epoch_time = time.time()
        idx_lst = [i for i in range(len(train_input_files))]
        random.shuffle(idx_lst)

        train_time = time.time()
        print('========================== Training part Begins ==========================')
        for train_step in range(N_steps):
            drawn_numbers, idx_lst = random_int(idx_lst, per_step)
            selected_input_files, selected_target_files1, selected_target_files2 = get_file_lst(drawn_numbers,
                                        train_input_files, train_target_files[0], train_target_files[1])
            pool = multiprocessing.Pool(processes=8)
            segments = pool.map(multi_sf_blocks.get_sf_blocks, [selected_input_files,
                                                               selected_target_files1,
                                                               selected_target_files2])
#             print('Number of 2-seconds long samples:', len(segments[0]))
            print('Expected iterations for train:', len(segments[0])//BATCH_SIZE - 1, end='\t')
            train_ds = generate_ds(segments[0], segments[1], segments[2])         
            
            train_sisnr = []
            for i, (input_batch, target_batch1, target_batch2) in enumerate(train_ds):
                loss = train_step(input_batch, target_batch1, target_batch2)
                train_sisnr.append(loss['train_sisnr'].numpy())
            print('.', end='')
            train_metrics.append(tf.reduce_mean(train_sisnr))
            with summary_writer.as_default():
                tf.summary.scalar('train_sisnr', tf.reduce_mean(train_sisnr), step=step*(epoch+1))
            print()
        print('==== Training part Done, for {} iterations, time taken: {:.2f} seconds ===='.format(i,
                                                                                time.time()-train_time))
        print()
        
        
        ## validation part
        val_idx_lst = [i for i in range(len(val_input_files))]
        random.shuffle(val_idx_lst)

        val_time = time.time()
        print('========================== Validation part Begins ==========================')
        for val_step in range(val_N_steps):
            drawn_numbers, val_idx_lst = random_int(val_idx_lst, per_step)
            val_selected_input_files, val_selected_target_files1, val_selected_target_files2 = get_file_lst(
                drawn_numbers, val_input_files, val_target_files[0], val_target_files[1])
            val_segments = pool.map(multi_sf_blocks.get_sf_blocks, [val_selected_input_files,
                                                                   val_selected_target_files1,
                                                                   val_selected_target_files2])
#             print('Number of 2-seconds long samples:', len(val_segments[0]))
            print('Expected iterations for validation:', len(val_segments[0])//BATCH_SIZE - 1)
            val_ds = generate_ds(val_segments[0], val_segments[1], val_segments[2])

            val_sisnr = []
            for j, (val_input_batch, val_target_batch1, val_target_batch2) in enumerate(val_ds):
                val_loss = test_step(val_input_batch, val_target_batch1, val_target_batch2)
                val_sisnr.append(val_loss['val_sisnr'].numpy())
            print(',', end='')
                
            val_metrics.append(tf.reduce_mean(val_sisnr))
            with summary_writer.as_default():
                tf.summary.scalar('val_sisnr', tf.reduce_mean(val_sisnr), step=val_step*(epoch+1))
            print()
        print('==== Validation part Done, for {} iterations, time taken: {:.2f} seconds ===='.format(j,
                                                                                    time.time()-val_time))

        logs = {'train_sisnr': tf.reduce_mean(train_metrics),
           'val_sisnr': tf.reduce_mean(val_metrics)}
        callbacks.on_epoch_end(epoch, logs)

        print('############### End of Epoch Num {}, time taken: {:.2f} seconds ###############'.format(epoch + 1, 
                                                                                    time.time() - epoch_time))
        print()
        print()

################### Start of Epoch Num 1, out of 3 Epochs ###################
========================== Training part Begins ==========================
Number of 2-seconds long samples: 46
Expected iterations for train: 4
...
Number of 2-seconds long samples: 46
Expected iterations for train: 4
...
Number of 2-seconds long samples: 53
Expected iterations for train: 5
...
==== Training part Done, for 2 iterations, time taken: 241.58 seconds ====

========================== Validation part Begins ==========================
Number of 2-seconds long samples: 23
Expected iterations for validation: 1
,,
Number of 2-seconds long samples: 22
Expected iterations for validation: 1
,,
Number of 2-seconds long samples: 18
Expected iterations for validation: 1
,,
==== Validation part Done, for 1 iterations, time taken: 57.74 seconds ====

Epoch 00001: saving model to C:/Users/Phil/anaconda3/envs/CTnet\checkpoints\cp-001.ckpt
############### End of Epoch Num 1, time taken: 311.95 seconds ##########

In [ ]:
latest = tf.train.latest_checkpoint("C:/Users/Phil/anaconda3/envs/CTnet/checkpoints")
latest

In [ ]:
# cv_tasnet.set_weights()로는 안 됨
cv_tasnet.load_weights(latest)